In [32]:
import tkinter as tk
from tkinter import messagebox
import pyodbc
import pandas as pd
import os
from datetime import datetime
import shutil

DATA_SOURCES = [
    {
        "directory": r'\\Azdwdflc01\DWH_IMPORT\PIM Data',
        "archive_folder":  r'\\Azdwdflc01\DWH_IMPORT\PIM Data\Archive\PIMData',
        "table_name": "files.PimData_02",
        "source": None,
        "file_keyword": 'pim',
        "truncate": True  # <<< Only truncate this one       #0
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\ME DATA",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\ME DATA\Archieve",
        "table_name": "Files.ME_SalesMarginLine_01",
        "source": "ME_Sales",
        "file_keyword": "sales",
        "truncate": False                                      #1
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\ME DATA",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\ME DATA\Archieve",
        "table_name": "Files.ME_PO_Data_02",
        "source": "ME_Sales",
        "file_keyword": "purchas",
        "truncate": False                                  #2
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\GH Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\GH Data\Archive",
        "table_name": "Files.GH_SalesMarginLines_01",
        "source": "GH",
        "file_keyword": "sales",
        "truncate": False                                 #3
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\GH Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\GH Data\Archive",
        "table_name": "Files.GH_PurchasingData_02",
        "source": "GH",
        "file_keyword": "purchas",
        "truncate": False                                #4
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\OEM Korea",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\OEM Korea\Archieve",
        "table_name": "Files.OEM_Sales_Korea_01",
        "source": "EXCEL_KOREA",
        "file_keyword": "sales",
        "truncate": False                                #5
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\OEM Korea",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\OEM Korea\Archieve",
        "table_name": "Files.OEM_PO_Korea_01",
        "source": "EXCEL_KOREA",
        "file_keyword": "purchas",
        "truncate": False                              #6
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\OEM EU",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\OEM EU\Archive",
        "table_name": "Files.OEM_EU_MarginLines_01",
        "source": "SAP_EXCEL",
        "file_keyword": "sales",
        "truncate": False                             #7
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\OEM EU",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\OEM EU\Archive",
        "table_name": "Files. SAP_OEM_PurchasingData_01",
        "source": "SAP_EXCEL",
        "file_keyword": "ZZZMM_PDC",
        "truncate": False                           #8
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data\Archive",
        "table_name": "Files.Sales_Peru_01",
        "source": "EXCEL_KOREA",
        "file_keyword": "Sales Data Peru",
        "truncate": False                         #9
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data\Archive",
        "table_name": "Files.PO_PERU _01",
        "source": "EXCEL_KOREA",
        "file_keyword": "Purchasing Data Peru",
        "truncate": False                       #10
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data\Archive",
        "table_name": "Files.Sales_Colombia_01",
        "source": "EXCEL_KOREA",
        "file_keyword": "sales",    #"Sales Data Colombia",
        "truncate": False                        #11
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data\Archive",
        "table_name": "Files.PO_Colombia_01",
        "source": "EXCEL_KOREA",
        "file_keyword": "Purchasing Data Colombia",
        "truncate": False                       #12
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\RYCO Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\RYCO Data\Archive",
        "table_name": "Files.RH_SalesData_01",
        "source": "EXCEL_KOREA",
        "file_keyword": "RycoMonthlySalesData",
        "truncate": False                      #13
    },
    {
        "directory": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "archive_folder": r"\\Azdwdflc01\DWH_IMPORT\PeruColombia Data\Archive",
        "table_name": "Files.RH_PurchasingData_01",
        "source": "EXCEL_KOREA",
        "file_keyword": "Purchasing Data",
        "truncate": False                      #14
    }
]

##=========================================================
def process_excel_file(config):
    directory = config["directory"]
    archive_folder = config["archive_folder"]
    table_name = config["table_name"]
    source = config["source"]
    file_keyword = config['file_keyword']
    truncate = config.get("truncate", False)

    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=azdwdflc01;'
        r'DATABASE=ODS;'
        r'Trusted_Connection=yes;'
    )
    cnxn = pyodbc.connect(conn_str)
    cursor = cnxn.cursor()

    files = [f for f in os.listdir(directory) if file_keyword.lower() in f.lower() and f.endswith(".xlsx")]

    if not files:
        messagebox.showerror("Error", f"No files containing '{file_keyword}' found.")
        return

    file_path = os.path.join(directory, files[0])
    df = pd.read_excel(file_path)

    # Clean and prepare data
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col].fillna('', inplace=True)
        else:
            df[col].fillna(0, inplace=True)

    if source:
        df['Source'] = source
    df['InsertDateTime'] = datetime.now()

    # Truncate before insert (if needed)
    if truncate:
        cursor.execute(f'TRUNCATE TABLE {table_name};')
        cnxn.commit()

    # Generate insert query dynamically
    columns = list(df.columns)
    safe_columns = [f'[{col}]' if ' ' in col else col for col in columns]

    insert_query = f'''
        INSERT INTO {table_name}
        ({', '.join(safe_columns)})
        VALUES ({', '.join(['?' for _ in columns])})
    '''
    rows = df.values.tolist()
    cursor.executemany(insert_query, rows)

    cnxn.commit()
    cnxn.close()

    # Archive file
    if not os.path.exists(archive_folder):
        os.makedirs(archive_folder)

    archive_path = os.path.join(archive_folder, files[0])
    shutil.move(file_path, archive_path)

    messagebox.showinfo("Success", f"File '{files[0]}' inserted and archived.")


#-----
def execute_PimData():
    process_excel_file(DATA_SOURCES[0])

def execute_ME_Sales_File():
    process_excel_file(DATA_SOURCES[1])

def execute_ME_Purchasing_File():
    process_excel_file(DATA_SOURCES[2])

def execute_GH_Sales_File():
    process_excel_file(DATA_SOURCES[3])

def execute_GH_Purchasing_File():
    process_excel_file(DATA_SOURCES[4])
    
def execute_EXCEL_KOREA_Sales_File():
    process_excel_file(DATA_SOURCES[5])
    
def execute_EXCEL_KOREA_Purchasing_File():
    process_excel_file(DATA_SOURCES[6])

def execute_OEM_SAP_Sales_File():
    process_excel_file(DATA_SOURCES[7])

def execute_OEM_SAP_Purchasing_File():
    process_excel_file(DATA_SOURCES[8])

def execute_Peru_Sales_File():
    process_excel_file(DATA_SOURCES[9])

def execute_Peru_Purchasing_File():
    process_excel_file(DATA_SOURCES[10])

def execute_Colombia_Sales_File():
    process_excel_file(DATA_SOURCES[11])

def execute_Colombia_Purchasing_File():
    process_excel_file(DATA_SOURCES[12])

def execute_RYCO_Sales_File():
    process_excel_file(DATA_SOURCES[13])

def execute_RYCO_Purchasing_File():
    process_excel_file(DATA_SOURCES[14])




# ====================================================================================================================================

# Create tkinter window
window = tk.Tk()
window.title("FLC Data Loader")
window.geometry("550x800")

# === PIM Button (centered) ===

frame_pim = tk.Frame(window)
frame_pim.grid(row=0, column=0, columnspan=2, pady=20)

label_file_path_pim = tk.Label(frame_pim, text="\\Azdwdflc01\\DWH_IMPORT\\PIM Data", fg="lightblue", font=("tahoma", 7))
label_file_path_pim.pack(side='top', pady=2)

button_pim = tk.Button(frame_pim, text="Truncate & Load PIM File", width=30, command=execute_PimData)
button_pim.pack(side='top', pady=2)

label_url_pim = tk.Label(frame_pim, text="--> files.PimData_01", fg="gray", font=("tahoma", 8))
label_url_pim.pack(side='top', pady=2)


# === 14 Other Buttons ===
# List your labels and functions as needed
button_labels = [
    "Load ME Sales File", "Load ME Purchasing File", 
    "Load GH Sales File", "Load GH Purchasing File",
    "Load OEM Korea Sales File", "Load OEM Korea Purchasing File",
    "Load OEM SAP Sales File", "Load OEM SAP Purchasing File",
    "Load Peru Sales File", "Load Peru Purchasing File",
    "Load Colombia Sales File", "Load Colombia Purchasing File",
    "Load RYCO Sales File", "Load RYCO Purchasing File"
]

row = 1
col = 0

for i, label in enumerate(button_labels):
    btn = tk.Button(window, text=label, width=25, command=lambda l=label: execute_button(l))
    btn.grid(row=row, column=col, padx=10, pady=10)
    if col == 0:
        col = 1
    else:
        col = 0
        row += 1

# Make both columns expand equally
window.grid_columnconfigure(0, weight=1)
window.grid_columnconfigure(1, weight=1)

#======================

buttons_info = [
    {
        "text": "Load ME Sales File",
        "command": execute_ME_Sales_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\ME DATA",
        "target_file": "--> Files.ME_SalesMarginLine_01"      #1
    },
    {
        "text": "Load ME Purchasing File",
        "command": execute_ME_Purchasing_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\ME DATA",
        "target_file": "-->  Files.ME_PO_Data_02"            #2
    },
    {
        "text": "Load GH Sales File",
        "command": execute_GH_Sales_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\GH Data",
        "target_file": "-->  Files.GH_SalesMarginLines_01"             #3
    },
       {
        "text": "Load GH Purchasing File",
        "command": execute_GH_Purchasing_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\GH Data",
        "target_file": "-->  Files.GH_SalesMarginLines_01"             #4
    },
     {
        "text": "Load OEM Korea Sales File",
        "command": execute_EXCEL_KOREA_Sales_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\OEM Korea",
        "target_file": "-->  Files.OEM_Sales_Korea_01"                 #5
    },
     {
        "text": "Load OEM Korea Purchasing File",
        "command": execute_EXCEL_KOREA_Purchasing_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\OEM Korea",
        "target_file": "-->  Files.OEM_PO_Korea_01"                    #6
    },
     {
        "text": "Load OEM SAP Sales File",
        "command": execute_OEM_SAP_Sales_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\OEM EU",
        "target_file": "-->  Files.OEM_EU_MarginLines_01"              #7
    },
     {
        "text": "Load OEM SAP Purchasing File",
        "command": execute_OEM_SAP_Purchasing_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\OEM EU",
        "target_file": "-->  Files. SAP_OEM_PurchasingData_01"           #8
    },
     {
        "text": "Load Peru Sales File",
        "command": execute_Peru_Sales_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "target_file": "-->  Files.Sales_Peru_01"                           #9
    },
     {
        "text": "Load Peru Purchasing File",
        "command": execute_Peru_Purchasing_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "target_file": "-->  Files.PO_PERU_01"                                #10
    },
     {
        "text": "Load Colombia Sales File",
        "command": execute_Colombia_Sales_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "target_file": "-->  Files.Sales_Colombia_01"                          #11
    },
     {
        "text": "Load Colombia Purchasing File",
        "command": execute_Colombia_Purchasing_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "target_file": "-->  Files.PO_Colombia_01"                               #12
    },
     {
        "text": "Load RYCO Sales File",
        "command": execute_RYCO_Sales_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\RYCO Data",
        "target_file": "-->  Files.RH_SalesData_01"                           #13
    },
     {
        "text": "Load RYCO Purchasing File",
        "command": execute_RYCO_Purchasing_File,
        "file_path": "\\Azdwdflc01\DWH_IMPORT\PeruColombia Data",
        "target_file": "-->  Files.RH_PurchasingData_01"                         #14
    }
    # Add other 12 button configs here
]


row = 1
col = 0

for info in buttons_info:
    frame = tk.Frame(window)
    frame.grid(row=row, column=col, padx=10, pady=10, sticky="n")

    label_path = tk.Label(frame, text=info["file_path"], fg="lightblue", font=("tahoma", 7))
    label_path.pack(side='top', pady=2)

    btn = tk.Button(frame, text=info["text"], width=25, command=info["command"])
    btn.pack(side='top', pady=2)

    label_target = tk.Label(frame, text=info["target_file"], fg="gray", font=("tahoma", 8))
    label_target.pack(side='top', pady=2)

    if col == 0:
        col = 1
    else:
        col = 0
        row += 1


#=============================================
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\admin.nasim.FLCLAN\AppData\Local\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\local_admin.nasim.FLCLAN1\Temp\ipykernel_29524\1658543320.py", line 244, in execute_RYCO_Purchasing_File
    process_excel_file(DATA_SOURCES[14])
  File "C:\Users\local_admin.nasim.FLCLAN1\Temp\ipykernel_29524\1658543320.py", line 185, in process_excel_file
    cursor.executemany(insert_query, rows)
pyodbc.ProgrammingError: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'Source'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'InsertDateTime'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")
